In [1]:
import heapq

def heuristic(state, target):
    return abs(sum(state) - target)

def a_star(n, M, capacities):
    start = tuple([0]*n)
    goal = M
    open_list = []
    heapq.heappush(open_list, (0 + heuristic(start, goal), 0, "", start))
    closed_list = set()

    while open_list:
        _, g, operations, state = heapq.heappop(open_list)

        if sum(state) == goal:
            return operations

        for i in range(n):
            for j in range(n):
                if i != j:
                    transfer = min(state[i], capacities[j] - state[j])
                    if transfer > 0:
                        new_state = list(state)
                        new_state[i] -= transfer
                        new_state[j] += transfer
                        new_state = tuple(new_state)
                        if new_state not in closed_list:
                            operation = f"Chuyển/Múc {transfer} lít nước từ gáo {i+1} qua gáo {j+1}"
                            heapq.heappush(open_list, (g + 1 + heuristic(new_state, goal), g + 1, operations + "\n" + operation, new_state))
                            closed_list.add(new_state)

        for i in range(n):
            if state[i] < capacities[i]:
                new_state = list(state)
                new_state[i] = capacities[i]
                new_state = tuple(new_state)
                if new_state not in closed_list:
                    operation = f"Chuyển/Múc {capacities[i]} lít nước từ bờ sông qua gáo {i+1}"
                    heapq.heappush(open_list, (g + 1 + heuristic(new_state, goal), g + 1, operations + "\n" + operation, new_state))
                    closed_list.add(new_state)

            if state[i] > 0:
                new_state = list(state)
                new_state[i] = 0
                new_state = tuple(new_state)
                if new_state not in closed_list:
                    operation = f"Vứt bỏ {state[i]} lít nước từ gáo {i+1}"
                    heapq.heappush(open_list, (g + 1 + heuristic(new_state, goal), g + 1, operations + "\n" + operation, new_state))
                    closed_list.add(new_state)

    return "Không có đáp án"
print(a_star(3, 15, [7, 8, 9]))



Chuyển/Múc 9 lít nước từ bờ sông qua gáo 3
Chuyển/Múc 7 lít nước từ bờ sông qua gáo 1
Chuyển/Múc 8 lít nước từ gáo 3 qua gáo 2
Vứt bỏ 1 lít nước từ gáo 3


In [ ]:
Trong thuật toán A*, hàm heuristic (hàm h') được sử dụng để ước lượng khoảng cách từ trạng 
thái hiện tại đến trạng thái mục tiêu. Hàm heuristic đóng vai trò quan trọng trong việc xác định 
chi phí ưu tiên khi tìm kiếm đường đi tốt nhất trong không gian trạng thái. 
- Hàm heuristic thường được thiết kế để đưa ra một ước lượng tối ưu về khoảng cách từ trạng thái 
hiện tại đến trạng thái mục tiêu mà không yêu cầu tính toán chi tiết tất cả các bước trung gian
- Trong bài toán này, hàm h' được thiết kế để đánh giá sự khác biệt giữa tổng lượng nước hiện tại trong các gáo và lượng nước mục tiêu cần đạt. Hàm h(state, target) trả về giá trị tuyến tính của sự chênh lệch giữa tổng lượng nước hiện tại trong các gáo và lượng nước mục tiêu. Ví dụ, nếu trạng thái hiện tại là (5, 3, 2) và mục tiêu là 13, thì hàm h sẽ trả về giá trị 3, thể hiện khoảng cách giữa trạng thái hiện tại và trạng thái mục tiêu. Hàm h' không xem xét chi phí di chuyển cụ thể giữa các trạng thái, nhưng nó giúp hướng dẫn thuật toán A* tìm kiếm theo hướng có thể dẫn đến lời giải tối ưu trong thời gian ngắn nhất.


In [5]:
import numpy as np
import pandas as pd
from numpy import genfromtxt
from sklearn.model_selection import train_test_split
def softmax(z):
    e_z = np.exp(z - np.max(z, axis=0))
    return e_z / np.sum(e_z, axis=0)
def logistic_softmax_regression(X, y, w_init, alpha, tol=1e-4, loop=10000):
    w = [w_init]
    N = X.shape[1]
    d = X.shape[0]
    K = len(np.unique(y))
    count = 0
    check_w = 20
    while count < loop:
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = np.eye(K)[:, y[i]].reshape(K, 1)
            zi = softmax(np.dot(w[-1].T, xi))
            w_new = w[-1] + alpha * np.dot(xi, (yi - zi).T)
            count += 1
            if count % check_w == 0:
                if np.linalg.norm(w_new - w[-check_w]) < tol:
                    return w
            w.append(w_new)
    return w
test = []
with open("output.csv") as f:
    for val in f.readlines():
        l = val.strip()
        spl = l.split(",")
        if len(spl) > 1:
            test.append([float(i) for i in spl[:]])
test = np.array(test).T
test = np.vstack((np.ones((1, test.shape[1])), test))
x = []
y = []
with open("input.csv") as f:
 for val in f.readlines():
    l = val.strip()
    spl = l.split(",")
    if len(spl) > 1:
        x.append([float(i) for i in spl[:-1]])
        if spl[-1] == "Iris-setosa":
            y.append(0)
        if spl[-1] == "Iris-versicolor":
            y.append(1)
        if spl[-1] == "Iris-virginica":
            y.append(2)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=12, stratify=y)
x = np.array(x).T
y = np.array(y)
x = np.vstack((np.ones((1, x.shape[1])), x))
w = logistic_softmax_regression(x, y, np.zeros((x.shape[0], len(np.unique(y)))), 
alpha=0.1)
y_train_pred = np.argmax(softmax(np.dot(w[-1].T, x)), axis=0)
y_train_pred
y_test_pred = np.argmax(softmax(np.dot(w[-1].T, test)), axis=0)
# print(y_test_pred)
for i in range(len(y_test_pred)):
    if y_test_pred[i] == 0:
        print('Iris-setosa')
    elif y_test_pred[i] == 1:
        print('Iris-versicolor')
    else:
        print('Iris-virginica')


Iris-setosa
Iris-setosa
Iris-setosa
Iris-setosa
Iris-setosa
Iris-setosa
Iris-setosa
Iris-setosa
Iris-setosa
Iris-setosa
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-virginica
Iris-virginica
Iris-virginica
Iris-virginica
Iris-virginica
Iris-virginica
Iris-virginica
